## Protein Processing Filteration

In [34]:
import pandas as pd
import numpy as np 

### Files to read|

In [35]:
filesList = ['IPIgGMS2','IPMWMS2']
# Read the control and experiment datasets
control_data = pd.read_excel('IPIgGMS2.xls')
experiment_data = pd.read_excel('IPMWMS2.xls')

#Display first 5 rows to know if data is correctly loaded.
print(control_data.head(5))
print(experiment_data.head(5))
print(control_data.info())

  accession        entry  coverage  proteinProbability  totalPeptides  \
0    P63268   ACTH_MOUSE     66.22                 1.0             22   
1    P60710   ACTB_MOUSE     70.67                 1.0             26   
2    P63260   ACTG_MOUSE     70.67                 1.0             25   
3    P68134   ACTS_MOUSE     66.05                 1.0             22   
4    Q8BFZ3  ACTBL_MOUSE     42.02                 1.0             13   

   uniquePeptides  razorPeptides  totalSpectralCount  uniqueSpectralCount  \
0               1              9                 287                    6   
1               2             26                 349                   34   
2               1              1                 324                    9   
3               1              3                 267                    5   
4               5              5                 120                   15   

   razorSpectralCount  ...  uniqueIntensity  razorIntensity  \
0                 145  ...     6.63

## Primary Screening

In [47]:
# Extract the 'entry' column from each DataFrame
control_subset = control_data[['entry', 'coverage', 'totalPeptides']]
experiment_subset = experiment_data[['entry', 'coverage', 'totalPeptides']]

#Difference
# Find proteins present in the experiment dataset but not in the control dataset
missing_proteins = experiment_proteins[~experiment_subset['entry'].isin(control_subset['entry'])]
print(missing_proteins)

#Similarity 
# Finding Proteins present in Both Data Set.
similar_proteins = experiment_proteins[experiment_subset['entry'].isin(control_subset['entry'])]
print(similar_proteins)

            entry  coverage  totalPeptides
14    KV5A5_MOUSE     32.03              3
32    CALL3_MOUSE     22.15              3
55    KV1A1_MOUSE     14.04              2
63    TDIF2_MOUSE      1.06              1
99    K1C13_MOUSE     14.42              9
...           ...       ...            ...
1399   NFIX_MOUSE      2.25              1
1402   RT09_MOUSE      2.82              1
1403   LEG7_MOUSE      5.88              1
1404  CEP41_MOUSE      3.49              1
1405  CAZA3_MOUSE      2.34              1

[258 rows x 3 columns]
            entry  coverage  totalPeptides
0      IGKC_MOUSE     59.81              6
1      ACTB_MOUSE     70.40             25
2      ACTS_MOUSE     66.05             21
3      GCAB_MOUSE     21.79              8
4      ACTH_MOUSE     66.22             21
...           ...       ...            ...
1389    COL_MOUSE      7.08              1
1390   BST2_MOUSE      5.23              1
1395  S10AB_MOUSE      7.14              1
1400  UBP19_MOUSE      0.88   

## Saving into two new Files on sheet 1

In [48]:
# Create a Pandas Excel writer object
with pd.ExcelWriter('ScreenedDataDifferenceMS2.xlsx') as writer:
    # Save the missing proteins data to the first sheet
    missing_proteins.to_excel(writer, sheet_name='Primary Screening', index=False)
    
print("ScreenedDataDifferenceMS2.xlsx is Saved")

# Create a Pandas Excel writer object
with pd.ExcelWriter('ScreenedDataSimilarMS2.xlsx') as writer:
    # Save the missing proteins data to the first sheet
    similar_proteins.to_excel(writer, sheet_name='Primary Screening', index=False)

print("ScreenedDataSimilarMS2.xlsx is saved.")
    


ScreenedDataDifferenceMS2.xlsx is Saved
ScreenedDataSimilarMS2.xlsx is saved.


## Secondary Screening

In [49]:
# Total Peptides >= 10 in missing proteins 
tPCount = missing_proteins[missing_proteins['totalPeptides'] >=10]
missing_proteins = tPCount.sort_values(by='coverage',ascending=False)
print(missing_proteins)

#to save into new sheet of same excel file
with pd.ExcelWriter('ScreenedDataDifferenceMS2.xlsx',engine='openpyxl', mode='a') as writer:
    # Save the missing proteins data to the first sheet
    missing_proteins.to_excel(writer, sheet_name='Secondary Screening', index=False)
    
print("ScreenedDataDifferenceMS2.xlsx is Saved")

           entry  coverage  totalPeptides
196   HG2A_MOUSE     36.92             10
412  SEP11_MOUSE     28.54             11
179  HSP72_MOUSE     17.85             10
103  K1C17_MOUSE     16.40             10
ScreenedDataDifferenceMS2.xlsx is Saved


In [55]:
#Ratio For Similar Proteins
similar_proteins.loc[:, 'Ratio Mw/Ipg'] = similar_proteins['totalPeptides'] *2

L:\anaconda\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
